In [1]:
!pip install selenium
!pip install beautifulsoup4

import os, random, sys, time
# from urllib.parse import urlparse
from selenium import webdriver
from bs4 import BeautifulSoup
import xlsxwriter
import pandas as pd
import csv

In [11]:
# Enter location of Chrome driver file on local machine
# If you don't have chrome driver download at: https://chromedriver.chromium.org/
# >>> stable release >>> select your appropriate OS download
browser = webdriver.Chrome("C:/Users/ashwi/Desktop/chromedriver.exe")
browser.get("https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")

#Below put your burner linkedin account login info...(must have >20 connections for code to work)
username = "@gmail.com"
password = ""

elementID = browser.find_element_by_id("username")
elementID.send_keys(username)
elementID = browser.find_element_by_id("password")
elementID.send_keys(password)
elementID.submit()


In [3]:
# enter name  of csv companies list file from crunchbase 
datafile = "companies-7-13-2021.csv"
data = list(csv.reader(open(datafile, encoding='utf-8')))

In [4]:
workbook = xlsxwriter.Workbook("processedCompanies.xlsx")
worksheet = workbook.add_worksheet()
row_num_excel = 1

In [5]:
def add_to_xlsx(list):
    for x in range(len(list)):
        worksheet.write(chr(x + 65) + str(row_num_excel), list[x])

In [6]:
def getNewProfileIDs(soup, profilesQueued):
    profilesID = []
#     pav = soup.find('section', {'class': 'artdeco-card pv-browsemap-section__wrapper ember-view'})
    pav = soup.find('ul', {'class': 'display-flex list-style-none flex-wrap'})
    all_links = pav.findAll('a', {'class':'ember-view'})
    for links in all_links:
        userID = links.get('href')
        if((userID not in profilesQueued)):
            profilesID.append(userID)
    return profilesID[0::2]

In [7]:
def getNewProfileDescriptions(soup, profilesQueued):
    profilesID = []
#     pav = soup.find('section', {'class': 'artdeco-card pv-browsemap-section__wrapper ember-view'})
    pav = soup.find('ul', {'class': 'display-flex list-style-none flex-wrap'})
    all_divs = pav.findAll('div', {'class':'lt-line-clamp lt-line-clamp--multi-line ember-view'})
    for divs in all_divs:
        description = str(divs.getText)
        intTemp = description.find("-webkit-line-clamp:")
        description = description[intTemp + 25:len(description)-17]
        profilesID.append(description)
        
        
    
    return profilesID

In [8]:
def getNewProfileNames(soup, profilesQueued):
    profilesID = []
#     pav = soup.find('section', {'class': 'artdeco-card pv-browsemap-section__wrapper ember-view'})
    pav = soup.find('ul', {'class': 'display-flex list-style-none flex-wrap'})
    all_divs = pav.findAll('div', {'class':'org-people-profile-card__profile-title t-black lt-line-clamp lt-line-clamp--single-line ember-view'})
    for divs in all_divs:
        name = str(divs.getText)
        name = name[157: len(name)-17]
        profilesID.append(name)
        
        
    
    return profilesID

In [9]:
def getListLinksDescription(linkedinLinks, profileDescriptions, names):
    joinedList = []
    for x in range(len(linkedinLinks)):
        temp = [linkedinLinks[x], profileDescriptions[x], names[x]]
        joinedList.append(temp)
    
    return joinedList

In [10]:
row = data[0]
row[3] = "Company LinkedIn"
row.append("Employee Title")
row.append("Employee LinkedIn")
row.append("Employee Name")
row.append("Strength Index")

add_to_xlsx(row)
row_num_excel += 1

for x in range(1, 4):
    try:
        row = data[x]
        print(row)
        add_to_xlsx(row)
        row_num_excel += 1

        time.sleep(2)
        companyLinkedInLink = row[3]
        lastChar = companyLinkedInLink[len(companyLinkedInLink)-1]

        if lastChar == "/":
            filteredPeopleLink = companyLinkedInLink + "people/?keywords=machine%20learning%20engineer"
        else:
            filteredPeopleLink = companyLinkedInLink + "/people/?keywords=machine%20learning%20engineer"

        browser.get(filteredPeopleLink)
        for x in range(0,2):
            time.sleep(2)
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        profilesQueued = []
        linkedinLinks = []
        profileDescriptions = []
        listLinksDescription = []

        linkedinLinks = getNewProfileIDs(BeautifulSoup(browser.page_source), profilesQueued)
        profileDescriptions = getNewProfileDescriptions(BeautifulSoup(browser.page_source), profilesQueued)
        names = getNewProfileNames(BeautifulSoup(browser.page_source), profilesQueued)

        listLinksDescription = getListLinksDescription(linkedinLinks, profileDescriptions, names)

        # words to check for strength Index must be added in lowercase
        strengthIndexWordCheck = ['machine', 'learning', 'ml', 'engineer', 'director', 'senior', 'vp', 'vice president']
        for x in listLinksDescription:
            weight = 0
            descriptionStr = x[1]
            for word in strengthIndexWordCheck:
                if word in descriptionStr.lower():
                    weight += 1
            x.append(weight)

        listLinksDescription = sorted(listLinksDescription, key = lambda x: x[3])
        listLinksDescription = listLinksDescription[::-1]



    #add information to excel sheet
        if len(listLinksDescription) < 5:
            numProfilesDisplayed = len(listLinksDescription)

        else:
            numProfilesDisplayed = 5

        for x in range(0, numProfilesDisplayed):
            listLinksDescription[x][0] = 'https://www.linkedin.com' + listLinksDescription[x][0]
            tempSpace = ["", "", "", ""]
            for y in range(0, len(listLinksDescription[x])):
                tempSpace.append(listLinksDescription[x][y])

            temp = tempSpace[5]
            tempSpace[5] = tempSpace[4]
            tempSpace[4] = temp

            add_to_xlsx(tempSpace)
            row_num_excel += 1
    except:
        print(">>>ERROR with company listed above...most likely company LinkedIn Link not provided from crunchbase excel file\n")
        
    time.sleep(10)
try:
    workbook.close()
except:
    print("processedCompanies.xlsx is open already...please close the file and run this program again")
print('finished!')

['Outreach', 'https://www.crunchbase.com/organization/outreach', 'https://www.outreach.io/', 'https://www.linkedin.com/company/outreach-saas/']
['DataRobot', 'https://www.crunchbase.com/organization/datarobot', 'http://datarobot.com', 'http://www.linkedin.com/company/datarobot']
['Olive', 'https://www.crunchbase.com/organization/olive-ai', 'https://www.oliveai.com', 'https://www.linkedin.com/company/hireolive/']
finished!
